<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Earth Analytics Education - EA  Python Course Spring 2021

## Important  - Assignment Guidelines

1. Before you submit your assignment to GitHub, make sure to run the entire notebook with a fresh kernel. To do this first, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart & Run All)
2. Always replace the `raise NotImplementedError()` code with your code that addresses the activity challenge. If you don't replace that code, your notebook will not run.

```
# YOUR CODE HERE
raise NotImplementedError()
```

3. Any open ended questions will have a "YOUR ANSWER HERE" within a markdown cell. Replace that text with your answer also formatted using Markdown.
4. **DO NOT RENAME THIS NOTEBOOK File!** If the file name changes, the autograder will not grade your assignment properly.
6. When you create a figure, comment out `plt.show()` to ensure the autograder can grade your plots. For figure cells, DO NOT DELETE the code that says `DO NOT REMOVE LINE BELOW`.

```
### DO NOT REMOVE LINE BELOW ###
student_plot1_ax = nb.convert_axes(plt)
```

* Only include the package imports, code, and outputs that are required to run your homework assignment.
* Be sure that your code can be run on any operating system. This means that:
   1. the data should be downloaded in the notebook to ensure it's reproducible
   2. all paths should be created dynamically using the `os.path.join`

## Follow to PEP 8 Syntax Guidelines & Documentation

* Run the `autopep8` tool on all cells prior to submitting (HINT: hit shift + the tool to run it on all cells at once!
* Use clear and expressive names for variables. 
* Organize your code to support readability.
* Check for code line length
* Use comments and white space sparingly where it is needed
* Make sure all python imports are at the top of your notebook and follow PEP 8 order conventions
* Spell check your Notebook before submitting it.

For all of the plots below, be sure to do the following:

* Make sure each plot has a clear TITLE and, where appropriate, label the x and y axes. Be sure to include UNITS in your labels.


### Add Your Name Below 
**Your Name:**

<img style="float: left;" src="colored-bar.png"/>

---

# Multispectral Remote Sensing II Homework

In this assignment, you will explore and quantify the impacts of fire
on the landscape using MODIS and Landsat remote sensing data. You will
calculate the Difference Normalize Burn Ratio using MODIS and Landsat
remote sensing data. You will also visually compare NAIP, Landsat
and MODIS data.

## Project  Data

To complete this assignment you will need to dowload the following data
using earthpy:

```python
# NAIP  post  fire
et.data.get_data('cs-test-naip')
# vector layer for cold springs fire
et.data.get_data('cold-springs-fire')
#  MODIS  pre/post  fire
et.data.get_data('cold-springs-modis-h4')
# Landsat pre/post fire
et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")
```

## Include the Plots, Text and Outputs As Described Below in this Notebook

For all plots:

* Add appropriate titles to your plot that clearly and concisely describe what the plot shows.
* Be sure to use the correct bands for each plot.
* Specify the source of the data used for each plot in a plot caption using `ax.text()`.


## Project Introduction (10 points)

Read the [overview of the cold springs fire.](https://www.earthdatascience.org/courses/use-data-open-source-python/data-stories/cold-springs-wildfire/). In the Markdown 
cell below, add a 2-4 sentence description of the Cold Springs Fire. This should 
include the event:

1. name, 
2. type, 
3. duration / dates and 
4. location. 

YOUR ANSWER HERE

![Colored Bar](colored-bar.png)

In [ ]:
# Autograding imports - do not modify this cell
import matplotcheck.notebook as nb
import matplotcheck.autograde as ag
import matplotcheck.raster as rs
import numpy as np

In [ ]:
# Import libraries (5 points)
# Only include imports required to run this notebook

# YOUR CODE HERE
raise NotImplementedError()

The cell below checks for your version of rioxarray, rasterio and
xarray.  You should ideally be running:

*  rioxarray >= 0.3.0
*  rasterio  >= 1.2.0

If you need to update your environment, run the following
in bash with your earth analytics python environment active:

`conda update -c conda-forge rioxarray`

`conda update -c conda-forge rasterio`

In [ ]:
# View package versions
rxr.show_versions()

In [ ]:
# DO NOT MODIFY THIS CELL
# Tests that the working directory is set to earth-analytics/data

path = os.path.normpath(os.getcwd())
student_wd_parts = path.split(os.sep)

if student_wd_parts[-2:] == ['earth-analytics', 'data']:
    print("\u2705 Great - it looks like your working directory is set correctly to ~/earth-analytics/data")
else:
    print("\u274C Oops, the autograder will not run unless your working directory is set to earth-analytics/data")

## Function To Open & Clean Landsat &  MODIS Data

###  TASK:  Clean  Up  Docstrings For a Function

Below you are given a function to process and clean Landsat and MODIS data. 
Add a numpy style docstring to the function with required **Parameters**
and **Returns** elements.

NOTE: You are welcome to modify the function code or simply use 
it as is. 

For all functions in this notebook, add a docstring that uses numpy style format (for examples, review the [Intro to Earth Data Sciene textbook chapter on Functions](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/functions-modular-code/write-functions-in-python/#docstring).

The docstring should include:

    * A one sentence description of what the function does.
    * Description of each input variable (parameter), following numpy docstring standards.
    * Description of each output object (return), following numpy docstring standards.

In [ ]:
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     variable=None):
    # YOUR CODE HERE
    raise NotImplementedError()
    """Open and clean a single landsat band .

    Parameters
    -----------
    band_path:string A path to the array to be opened
    crop_bound:geopandas GeoDataFrame
    A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
     A tuple of min and max range of values for the data. Default = None

    Returns
    -----------

    """

    crop_bound_box = [box(*crop_bound.bounds.loc[0])]

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True,
                                 variable=variable).rio.clip(crop_bound_box,
                                                             crs=crop_bound.crs,
                                                             all_touched=True,
                                                             from_disk=True).squeeze()
    except:
        raise ValueError(
            "Oops - I couldn't clip your data. This may be due to a crs error.")

    # Only mask the data to the valid range if a valid range tuple is provided
    if valid_range is not None:
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

    return band

![Colored Bar](colored-bar.png)

# Figure 1 -  3  Subplots Containing  Color InfraRed (CIR) Imagery -  for  NAIP, Landsat and MODIS

**NOTE:  You will be able to complete the MODIS subplot after next week's class!**

Create a single figure that contains a **vertical grid** of 3 subplots that 
show color infrared (also called false color) composite images using:

* Post Fire NAIP data (this is the data that you downloaded for your week 6 homework)
* Post Fire Landsat data (use: `et.data.get_data(url="https://ndownloader.figshare.com/files/21941085")`
* Post Fire MODIS data (use: `et.data.get_data('cold-springs-modis-h5')`

For each subplot, be sure to:

* Crop the data to the square fire boundary extent.
* Overlay the fire boundary layer (`vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`) on top of the image.
* Use the band combination **r = infrared band**, **g = red band**, **b = green** band.
* Label each plot with the data type (NAIP vs. Landsat vs. MODIS) and spatial resolution.

HINT: In a CIR image, the NIR band is plotted on the “red” band, the red band is plotted on the "green" band and the green band is plotted on the "blue" band.

[Click here to see what the final plot should look like.](https://www.earthdatascience.org/courses/earth-analytics-python/multispectral-remote-sensing-modis/)

In [ ]:
# Open fire boundary shapefile in this cell

# YOUR CODE HERE
raise NotImplementedError()

## Open Landsat Data

In the cells below, open and process your Landsat data using loops and string 
manipulation of paths. Your data processing output should be a cropped and 
masked **xarray DataArray** object for both the pre and post fire images. 
You will use this object later in the assignment. At the end of the cell below,  
call your post fire Landsat xarray object that's been cropped and cleaned.

*  Important: Set the valid range for your landsat data to be **0-10,000** which maps to an unscaled valid range of 0-1 (no reflectance to 100% reflectance).
*  Only open the set ofbands that you need to process nbr and your CIR image.
* Use the `open_clean_bands()` function above to create the 
cropped and cleaned  **xarray DataArrays** for each Landsat scene.
* Hint: This function can also be used to open your NAIP and MODIS data below.


In [ ]:
# Create loop to process both Landsat scenes in this cell
# Use the function open_clean_bands to process your data

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This cell is testing your data output above

student_landsat_arr = _

landsat_points = 0

# Ensure the data is stored in a dataframe.
if isinstance(student_landsat_arr, xr.DataArray):
    print('\u2705 Your data is stored in a DataArray!')
    landsat_points += 3
else:
    print('\u274C It appears your data is not stored in a DataArray. ',
          'To see what type of object your data is stored in, check its type with type(object)')

print("\n \u27A1 You received {} out of 3 points for creating your array.".format(
    landsat_points))
landsat_points

In [ ]:
# DO NOT MODIFY THIS CELL


## Open MODIS hdf4 Data

In the cells below, open and process your MODIS hdf4 data using loops and 
string manipulation of paths. Your workflow should produce a cropped and 
masked **xarray DataArray** for both the pre and post fire 
scenes which you will use later in the assignment. You should also create a 
plotting extent object for the MODIS imagery. 

At the end of your cell, call your post fire MODIS array that's been cropped and cleaned!  

Use the function `open_clean_bands()` included in this notebook to open and 
crop the MODIS data.

* MODIS data is hierarchical, and not stored in multiple files like Landsat. In order to only get the desired bands for the homework, you can use the `variable=` argument in the provided function. If you pass a list of just the desired band names, the function will return an xarray Dataset with just those bands. 


In [ ]:
# Process MODIS Data

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This cell is testing your data output above

student_modis_arr = _

modis_points = 0

# Ensure the data is stored in a dataframe.
if isinstance(student_modis_arr, xr.DataArray):
    print('\u2705 Your data is stored in a DataArray!')
    modis_points += 3
elif isinstance(student_modis_arr, xr.Dataset):
    print('\u2705 Your data is stored in a DataArray!')
    modis_points += 3
else:
    print('\u274C It appears your data is not stored in a DataArray. ',
          'To see what type of object your data is stored in, check its type with type(object)')

print("\n \u27A1 You received {} out of 3 points for creating your array.".format(
    modis_points))
modis_points

In [ ]:
# DO NOT MODIFY THIS CELL

## Process NAIP Post Fire Data

In the cell below, use  the `open_clean_bands` function to open and 
crop the post-fire cold springs fire NAIP data.

In [ ]:
# Process NAIP data

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This cell is testing your data output above

student_naip_arr = _

naip_points = 0

# Ensure the data is stored in a dataframe.
if isinstance(student_naip_arr, xr.DataArray):
    print('\u2705 Your data is stored in a DataArray!')
    naip_points += 3
else:
    print('\u274C It appears your data is not stored in a DataArray. ',
          'To see what type of object your data is stored in, check its type with type(object)')

print("\n \u27A1 You received {} out of 3 points for creating your array.".format(
    naip_points))
naip_points

In [ ]:
# DO NOT MODIFY THIS CELL

## Figure 1: Plot CIR for NAIP, Landsat and MODIS Using Post Fire Data (15 points each subplot)

In the cell below, create a figure with 3 subplots stacked vertically.

In each subplot, plot a CIR composite image using the post-fire data for:

* NAIP  (first figure axis) 
* Landsat (second figure axis) 
* MODIS (third figure axis)

Respectively on this figure. In order to plot this figure using `earthpy.plot_rgb()` 
you will need to turn your data into a masked numpy array using the `clean_array_plot` 
function.

### Plotting Xarray Data  Using EarthPy
(Extra information in case you are wondering about this extra step!)

The `clean_array_plot` function is needed to plot xarray DataArrays with `ep.plot_rgb()`. This has to do with how `ep.plot_rgb()` handles `null` values in xarray DataArrays. `ep.plot_rgb()` was built around plotting masked numpy arrays. Masked numpy arrays are composed of two arrays, one with the stored values, and one that is applied as a mask over the stored values. xarray DataArrays integrate the mask into the DataArray by replacing all of the values with `nan` values. For  plotting,  these
`nan` values need to  
be masked. 

The `clean_array_plot()` function below, masks `nan` values for you
so the data plot properly using `plot_rgb()`.

### Task (5 points): Add Numpy  Style  Docstring  to Function  Below
    
Add a numpy-style docstring to the function below.

In [ ]:
def clean_array_plot(xr_obj):
    # This function takes a single xarray object as an input and produces a
    # cleaned numpy array output for plotting
    # YOUR CODE HERE
    raise NotImplementedError()

    return ma.masked_array(xr_obj.values,  xr_obj.isnull())

In [ ]:
# Plot CIR of Post Fire NAIP, Landsat & MODIS together in one figure

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot01_CIR_res_comparison = nb.convert_axes(plt, which_axes="all")

In [ ]:
# DO NOT MODIFY THIS CELL

In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for NAIP subplot


In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for Landsat subplot


In [ ]:
# DO NOT TOUCH THIS CELL - autograding tests for MODIS subplot


![Colored Bar](colored-bar.png)

# Figure 2 Overview: Difference NBR (dNBR) Using Landsat  & MODIS Data (25 points each subplot)

Create a figure that has two subplots stacked vertically using the same MODIS and Landsat data that you processed above. 

* Subplot one: classified dNBR using Landsat data
* Subplot two: classified dNBR using MODIS data 

For each subplot, overlay the fire extent boundary `vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`
on top of the dNBR map

First, create a dNBR raster for Landsat and Modis, using the datasets you opened earlier. Then classify each dNBR raster.

To classify each dNBR raster, create a function `classify_dnbr()` that does this for you.

When you plot your MODIS data, you may notice that the data does not contain all of the classes that Landsat contains which can range from 1-5. To ensure that your colormap plots properly, set the `vmin=` and `vmax=` parameters to 1 and 5 respectively when you call `ep.plot_bands()`:

`vmin=1, vmax=5`


## Figure Legend

You only need one legend for this figure. The `ep.draw_legend()` function will create a legend of "boxes" if you provide it with an:

1. `imshow()` image object
2. classes : a list of numbers that represent the classes in your numpy array
3. titles: a list of dNBR class names example: `["High Severity", "Low Severity"]`

### dNBR Classes

Note: if you scaled your data, you may need to scale the values below by a factor of 10.

| SEVERITY LEVEL  | dNBR RANGE |
|----------------|--------------|
| Enhanced Regrowth | < -.1 |
| Unburned        | -.1 to +.1 |
| Low Severity     | +.1 to +.27 |
| Moderate Severity  | +.27 to +.66 |
| High Severity    | > .66 |


HINT: Your dNBR classification list should look like this:
`[-np.inf, -.1, .1, .27, .66, np.inf]`

HINT 2: If you want to use them, these are the colors used in the maps on the website:

`["g", "yellowgreen", "peachpuff", "coral", "maroon"]`

The code to create a custom colormap for the plot is below:

```
nbr_colors = ["g", "yellowgreen", "peachpuff", "coral", "maroon"]
nbr_cmap = ListedColormap(nbr_colors)
```

## Function classify_dnbr (5 points)

In the cell below, write a function called `classify_dnbr` that: 
1. Classifies a numpy array using classes/bins defined in the function. 
    * **1 input:**
        * 1) numpy array containing dNBR data in numpy array format 
2. Returns a classified numpy array. 
    * **1 output:**
        * 1) numpy array with classified values (integers)
        
HINT: If you are getting errors due to there being too many classes when trying to plot, it's likely because you have `null` values in your array being classified above or below your infinity values! This is confusing, but can happen when digitizing and array with `null` values in it. Set up your function so that the output is masked in the same places the input is masked to avoid this issue. There shouldn't be any values greater then 5 in your digitized array, much like there shouldn't be any values greater then 10000 in a Landsat array!

In [ ]:
# Add your function here. Do NOT modify the function name
def classify_dnbr(arr):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Calculate and classify dNBR for Landsat - be sure to use the correct bands!
# Call your classified array at the end of the cell

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This cell  is testing your data output above

student_landsat_dnbr_class = _

landsat_dnbr_points = 0

values, counts = np.unique(student_landsat_dnbr_class, return_counts=True)

correct_values = [1, 2, 3, 4, 5]

correct_counts = [34, 2781, 456, 940, 775]

if all(value in values.tolist() for value in correct_values):
    print("\u2705 All expected classes were found in the raster!")
    landsat_dnbr_points += 3
else:
    print("\u274C Not all expected classes were found in the raster, or too many classes were found.")

if all(count in counts.tolist() for count in correct_counts):
    print("\u2705 The correct amount of each class exists in the raster!")
    landsat_dnbr_points += 3
else:
    print("\u274C Each class does not have the correct amount of values in the raster. Make sure you are masking NA values!")

print("\n \u27A1 You received {} out of 6 points for creating your array.".format(
    landsat_dnbr_points))
landsat_dnbr_points

In [ ]:
# Calculate dNBR for MODIS - be sure to use the correct bands!
# Call your classified array at the end of the cell

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This cell  is testing your data output above

student_modis_dnbr_class = _

modis_dnbr_points = 0

values, counts = np.unique(student_modis_dnbr_class, return_counts=True)

correct_values = [2, 3, 4]
correct_counts = [2, 12, 15]

if all(value in values.tolist() for value in correct_values):
    print("\u2705 All expected classes were found in the raster!")
    modis_dnbr_points += 3
else:
    print("\u274C Not all expected classes were found in the raster, or too many classes were found.")

if all(count in counts.tolist() for count in correct_counts):
    print("\u2705 The correct amount of each class exists in the raster!")
    modis_dnbr_points += 3
else:
    print("\u274C Each class does not have the correct amount of values in the raster. Make sure you are masking NA values!")

print("\n \u27A1 You received {} out of 6 points for creating your array.".format(
    modis_dnbr_points))
modis_dnbr_points

In [ ]:
# Plot Difference NBR (dNBR) for Landsat & MODIS together in one figure

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot03_landsat_dnbr = nb.convert_axes(plt, which_axes="all")

In [ ]:
# Ignore this cell - autograding tests for Landsat subplot


In [ ]:
# Ignore this cell - autograding tests for MODIS subplot


![Colored Bar](colored-bar.png)

# Landsat vs MODIS  Burned Area (8 points)

In the cell below, print the total area burned in classes 4 and 5 (moderate to high severity) for both datasets 
(Landsat and MODIS). After your print statements, call your variables in a list at the end of the cell. The list should be in this order: 
```
burned_landsat_class_4_area, burned_landsat_class_5_area, burned_modis_class_4_area, burned_modis_class_5_area]
```

HINT: Feel free to experiment with loops to complete this part of the homework. 

In [ ]:
# Total Burned Area in Classes 4 and 5 for Landsat and MODIS

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

![Colored Bar](colored-bar.png)

# Bonus Plot: Difference NDVI (dNDVI) Using Landsat & MODIS Data (20 points each subplot)

Plot the NDVI difference using before and after Landsat and MODIS data 
that cover the Cold Springs Fire study area. For each axis be sure to:

1. overlay the fire boundary (`vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp`) on top of the data.
2. Be sure that the data are cropped using the fire boundary extent.

In the cell below, create a figure with 2 subplots stacked vertically:
* Plot dNDVI for Landsat on the first axis of the figure.
* Plot dNDVI using MODIS on the second axis of the figure.

Use the "before" and "after" data that you processed above to calculate NDVI difference for both MODIS and Landsat


## NDVI Difference

To create the NDVI Difference raster using "before" and "after" fire 
Landsat and MODIS data, you must first calculate NDVI for each 
dataset "before" and "after" the fire. 

Once you have the "before" and "after" NDVI arrays, you can subtract 
the pre-fire NDVI array FROM the post-fire NDVI array (post-fire minus pre-fire). 

The resulting array will show you change in the area's NDVI from the first image to the second image.

HINT: Remember, you can use `es.normalized_diff(band_1, band_2)` to get the NDVI of an image. 

In [ ]:
# Plot Difference NDVI for Landsat & MODIS together in one figure

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot02_landsat_modis_ndvi_diff = nb.convert_axes(plt, which_axes="all")

In [ ]:
# Ignore this cell - autograding tests for Landsat subplot


In [ ]:
# Ignore this cell - autograding tests for MODIS subplot


# Do not edit this cell! (5 points)

* Each figure specifies the source of the data (for each plot) using a plot caption created with `ax.text()`.

# Do not edit this cell! (5 points)

The notebook will also be checked for overall clean code requirements as specified at the **very top** of this notebook! Some of these requirements include (review the top cells for more specifics): 

* Notebook begins at cell [1] and runs on any machine in its entirety.
* PEP 8 format is applied throughout (including lengths of comment and code lines).
* No additional code or imports in the notebook
* Notebook is fully reproducible. This means:
   * reproducible paths using the os module.
   * data downloaded using code in the notebook.
   * all imports at top of notebook.

# Do not edit this cell!  (5 points)

All functions contain docstrings with inputs and outputs clearly identified and following numpy docstring standards.